# 세션 16 — CNN 모델 빌드 실습 (SVHN)

## 학습 목표
- CNN 구조 설계부터 실험까지 end-to-end 경험
- Feature Extractor와 Classifier 분리 설계
- Conv/Pool/ReLU 조합과 파라미터 계산
- Hook을 활용한 레이어 출력 shape 추적
- 하이퍼파라미터 튜닝 경험

## 데이터셋: SVHN (Street View House Numbers)
- Google Street View에서 수집한 집 번호 이미지
- 10개 클래스 (숫자 0-9)
- 32×32 컬러 이미지
- 학습: 73,257개, 테스트: 26,032개

---

In [ ]:
# 1. 시스템 업데이트 및 언어 관련 패키지 설치
# (실행 시 시간이 좀 걸릴 수 있어요!)
!sudo apt-get update -qq
!sudo apt-get install locales -qq

# 2. 한국어 (ko_KR.UTF-8) locale 생성
# 이 단계에서 오류가 나지 않아야 해요!
!sudo locale-gen ko_KR.UTF-8

# 3. 환경 변수 설정
# 파이썬 코드 안에서 실행합니다.
import os
os.environ['LANG'] = 'ko_KR.UTF-8'
os.environ['LC_ALL'] = 'ko_KR.UTF-8'
os.environ['LC_CTYPE'] = 'ko_KR.UTF-8'
os.environ['LANGUAGE'] = 'ko_KR.UTF-8'

# 4. 런타임 다시 시작 (!!!! 아주 중요합니다 !!!!)
# 이 셀을 실행한 후에는 반드시 콜랩 메뉴에서 런타임을 재시작해야 해요.
# 메뉴: "런타임(Runtime)" -> "런타임 다시 시작(Restart runtime)" 클릭!
# 재시작 후에는 이 위의 코드 셀들을 다시 실행할 필요 없어요.
# 바로 다음 단계로 넘어가시면 됩니다.

# 5. (선택 사항) 설정 확인 - 런타임 재시작 후 이 셀을 실행해보세요.
# 'ko_KR.UTF-8' 관련 내용이 보이면 성공!
# !locale

런타임 다시 시작

In [ ]:
# 나눔 폰트 설치 (Colab에서 한글 표시를 위해 가장 많이 사용돼요)
!sudo apt-get install -y fonts-nanum > /dev/null 2>&1
!sudo fc-cache -fv > /dev/null 2>&1

# Matplotlib 등에서 한글 폰트 설정을 위한 코드 (streamlit과는 직접 관련 없을 수도 있지만,
# 만약을 위해 환경 준비 차원에서 실행해주세요)
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

# 설치된 폰트 경로 확인
font_path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔바른고딕 예시
if os.path.exists(font_path):
    fm.fontManager.addfont(font_path)
    plt.rc('font', family='NanumBarunGothic')
    plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지
    print("한글 폰트 설정 완료: NanumBarunGothic")
else:
    print(f"Warning: 폰트 파일이 없습니다: {font_path}")

# (선택 사항) 시스템에 설치된 폰트 목록 확인
# [f.name for f in fm.fontManager.ttflist if 'Nanum' in f.name]

## Section 1: 환경 설정 및 라이브러리 임포트

In [ ]:
# PyTorch 및 관련 라이브러리 임포트
import torch  # PyTorch 메인 라이브러리
import torch.nn as nn  # 신경망 모듈 (레이어, 손실함수 등)
import torch.optim as optim  # 최적화 알고리즘 (Adam, SGD 등)
import torch.nn.functional as F  # 활성화 함수, Softmax 등
from torch.utils.data import DataLoader  # 데이터 로더 (배치 처리)

# torchvision: 컴퓨터 비전 관련 유틸리티
import torchvision  # 비전 관련 전체 모듈
import torchvision.transforms as transforms  # 이미지 전처리 및 증강
from torchvision import datasets  # SVHN, CIFAR 등 표준 데이터셋

# 데이터 처리 및 시각화
import numpy as np  # 수치 연산 라이브러리
import matplotlib.pyplot as plt  # 그래프 및 이미지 시각화
from collections import OrderedDict  # 순서가 보장되는 딕셔너리

# 진행 상황 표시 (선택사항)
from tqdm import tqdm  # 진행률 표시줄 (progress bar)

# GPU 사용 가능 여부 확인 및 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.cuda.is_available(): CUDA(GPU)가 사용 가능한지 확인
# GPU 있으면 'cuda', 없으면 'cpu' 사용
print(f'사용 디바이스: {device}')

# 재현성을 위한 시드 고정
torch.manual_seed(42)  # PyTorch 난수 시드 고정
np.random.seed(42)  # NumPy 난수 시드 고정
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)  # CUDA 난수 시드 고정
    torch.backends.cudnn.deterministic = True  # 결정적 알고리즘 사용
    torch.backends.cudnn.benchmark = False  # 벤치마크 비활성화 (재현성 우선)

print('환경 설정 완료!')
print(f'PyTorch 버전: {torch.__version__}')
print(f'Torchvision 버전: {torchvision.__version__}')

## Section 2: 데이터셋 로드 및 전처리

SVHN 데이터셋을 다운로드하고 전처리 파이프라인을 정의합니다.

In [ ]:
# 데이터 전처리 파이프라인 정의

# 학습용 전처리 (데이터 증강 포함)

# 코드 작성
transform_train = transforms.Compose([
    # RandomCrop: 이미지를 32x32 크기로 랜덤하게 크롭하되, 4픽셀 패딩 추가
    # 패딩으로 36x36이 되고, 여기서 32x32를 랜덤 위치에서 자름 (위치 변화)
    transforms.RandomCrop(32, padding=4),

    # RandomHorizontalFlip: 50% 확률로 이미지를 좌우 반전
    # 숫자 이미지에서 좌우 대칭은 의미가 있을 수 있음 (예: 6과 9는 다르지만)
    transforms.RandomHorizontalFlip(p=0.5),

    # ColorJitter: 밝기, 대비, 채도를 랜덤하게 변경 (조명 조건 변화 시뮬레이션)
    # brightness: 밝기를 ±20% 변경
    # contrast: 대비를 ±20% 변경
    # saturation: 채도를 ±20% 변경
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),

    # ToTensor: PIL Image를 PyTorch Tensor로 변환
    # (H, W, C) -> (C, H, W) 형태로 변경
    # 픽셀 값을 [0, 255] -> [0.0, 1.0] 범위로 정규화
    transforms.ToTensor(),

    # Normalize: 각 채널을 평균 0.5, 표준편차 0.5로 정규화
    # 실제로는 [0, 1] -> [-1, 1] 범위로 변환하는 효과
    # 공식: normalized = (pixel - mean) / std
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# 테스트용 전처리 (증강 없음)
transform_test = transforms.Compose([
    # ToTensor: 이미지를 텐서로 변환 (증강 없이)
    transforms.ToTensor(),

    # Normalize: 학습 데이터와 동일한 정규화 적용 (중요!)
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

print('데이터 전처리 파이프라인 정의 완료!')
print('\n학습용 전처리 단계:')
print('  1. RandomCrop(32, padding=4) - 위치 변화')
print('  2. RandomHorizontalFlip - 좌우 반전')
print('  3. ColorJitter - 색상 변화')
print('  4. ToTensor - 텐서 변환')
print('  5. Normalize - 정규화')
print('\n테스트용 전처리 단계:')
print('  1. ToTensor - 텐서 변환')
print('  2. Normalize - 정규화 (학습과 동일)')

In [ ]:
# SVHN 데이터셋 다운로드 및 로드

print('SVHN 데이터셋 다운로드 중...')
print('(처음 실행 시 다운로드에 시간이 걸릴 수 있습니다)\n')

# 학습 데이터셋 로드
train_dataset = datasets.SVHN(
    root='./data',  # 데이터를 저장할 디렉토리 경로
    split='train',  # 'train' 또는 'test' 선택
    download=True,  # 데이터가 없으면 자동 다운로드
    transform=transform_train  # 학습용 전처리 적용
)

# 테스트 데이터셋 로드
test_dataset = datasets.SVHN(
    root='./data',  # 동일한 디렉토리 사용
    split='test',  # 테스트 데이터 사용
    download=True,  # 없으면 다운로드
    transform=transform_test  # 테스트용 전처리 적용
)

# 데이터셋 정보 출력
print('='*60)
print('데이터셋 정보')
print('='*60)
print(f'학습 데이터: {len(train_dataset):,}개')
print(f'테스트 데이터: {len(test_dataset):,}개')
print(f'클래스 수: 10개 (숫자 0-9)')
print(f'이미지 크기: 32×32×3 (컬러)')
print('='*60)

In [ ]:
# 데이터 로더 생성
# DataLoader: 배치 단위로 데이터를 로드하고, 셔플링, 병렬 처리 등을 수행

# 배치 크기 설정
batch_size = 128  # 한 번에 처리할 이미지 개수
# 배치 크기가 클수록: 학습 속도 빠름, GPU 메모리 많이 사용
# 배치 크기가 작을수록: 학습 불안정, GPU 메모리 적게 사용

# 학습용 데이터 로더
# 코드 작성
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True

)

# 테스트용 데이터 로더
test_loader = DataLoader(
    test_dataset,  # 테스트 데이터셋
    batch_size=batch_size,  # 배치 크기 (평가 시에는 더 크게 해도 됨)
    shuffle=False,  # 테스트 시에는 순서를 섞지 않음
    num_workers=2,
    pin_memory=True
)

# 데이터 로더 정보 출력
print('\n데이터 로더 생성 완료!')
print(f'\n학습 데이터 로더:')
print(f'  - 배치 크기: {batch_size}')
print(f'  - 배치 수: {len(train_loader)}개')
print(f'  - 총 샘플: {len(train_dataset):,}개')
print(f'\n테스트 데이터 로더:')
print(f'  - 배치 크기: {batch_size}')
print(f'  - 배치 수: {len(test_loader)}개')
print(f'  - 총 샘플: {len(test_dataset):,}개')

In [ ]:
# 샘플 이미지 시각화
# 데이터셋이 제대로 로드되었는지 확인

# 클래스 이름 (SVHN은 0-9 숫자)
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# 학습 데이터에서 16개 샘플 가져오기
# 코드 작성
images, labels = next(iter(train_loader))

# 데이터 로더를 이터레이터로 변환
# 첫 번째 배치 가져오기

# 이미지를 [-1, 1] 범위에서 [0, 1]로 변환 (시각화용)
def denormalize(tensor):
    # 정규화 역변환: pixel = (normalized * std) + mean
    # 우리는 mean=0.5, std=0.5로 정규화했으므로
    # 코드 작성
    return tensor * 0.5 + 0.5

# 16개 이미지를 4x4 그리드로 시각화
plt.figure(figsize=(10, 10))
for i in range(16):
    plt.subplot(4, 4, i+1)  # 4x4 그리드의 i+1번째 위치

    # 이미지를 (C, H, W) -> (H, W, C) 형태로 변환
    img = denormalize(images[i]).cpu().numpy().transpose(1, 2, 0)

    # 이미지 표시
    plt.imshow(img)
    plt.title(f'Label: {class_names[labels[i]]}')  # 레이블 표시
    plt.axis('off')  # 축 숨기기

plt.tight_layout()  # 레이아웃 자동 조정
plt.suptitle('SVHN 학습 데이터 샘플', y=1.02, fontsize=16)
plt.show()

print('\n샘플 이미지 시각화 완료!')
print('SVHN 데이터는 실제 Street View에서 수집된 집 번호 이미지입니다.')
print('다양한 조명, 각도, 배경을 가지고 있어 실전적인 학습이 가능합니다.')

## Section 3: CNN 모델 정의

Feature Extractor와 Classifier를 분리하여 설계합니다.

In [ ]:
# CNN 모델 클래스 정의
# Feature Extractor와 Classifier를 명확히 분리

class SVHN_CNN(nn.Module):
    """
    SVHN 숫자 분류를 위한 CNN 모델

    구조:
        Feature Extractor:
            - Conv Block 1: Conv(32) + ReLU + MaxPool
            - Conv Block 2: Conv(64) + ReLU + MaxPool
            - Conv Block 3: Conv(128) + ReLU + MaxPool

        Classifier:
            - Flatten
            - FC1: 2048 -> 512
            - FC2: 512 -> 10
    """

    def __init__(self, num_classes=10):
        """
        모델 초기화

        Args:
            num_classes (int): 출력 클래스 수 (기본값: 10)
        """
        super(SVHN_CNN, self).__init__()  # nn.Module 초기화

        # ===== Feature Extractor =====
        # 입력 이미지에서 유용한 특징을 추출하는 부분

        # Conv Block 1: 입력 채널 3 (RGB) -> 출력 채널 32
        self.conv1 = nn.Conv2d(
          # 코드 작성
          in_channels=3, # RGB
          out_channels=32, # 출력 채널 수 (필터 개수)
          kernel_size=3, # 필터 크기 3*3
          padding=1 # 출력 크기 유지: 32*32
        )
        # 파라미터 수: (3*3*3 + 1) * 32 = 28 * 32 = 896개
        # 출력 크기: [batch, 32, 32, 32]

        # MaxPool1: 공간 해상도 절반으로 축소
        self.pool1 = nn.MaxPool2d(
          # 코드 작성
          kernel_size=2, # 2*2 영역에서 최대값 선택
          stride=2       # 2칸씩 이동

        )
        # 출력 크기: [batch, 32, 16, 16]

        # Conv Block 2: 32 채널 -> 64 채널
        self.conv2 = nn.Conv2d(
            in_channels=32,     # 이전 레이어 출력 채널
            out_channels=64,    # 출력 채널 수 (더 많은 필터)
            kernel_size=3,
            padding=1
        )
        # 파라미터 수: (3*3*32 + 1) * 64 = 289 * 64 = 18,496개
        # 출력 크기: [batch, 64, 16, 16]

        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 출력 크기: [batch, 64, 8, 8]

        # Conv Block 3: 64 채널 -> 128 채널
        self.conv3 = nn.Conv2d(
            in_channels=64,
            out_channels=128,   # 가장 높은 수준의 특징 추출
            kernel_size=3,
            padding=1
        )
        # 파라미터 수: (3*3*64 + 1) * 128 = 577 * 128 = 73,856개
        # 출력 크기: [batch, 128, 8, 8]

        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        # 출력 크기: [batch, 128, 4, 4]
        # Flatten 후: [batch, 128*4*4] = [batch, 2048]

        # ===== Classifier =====
        # 추출된 특징을 바탕으로 최종 분류

        # Fully Connected Layer 1: 2048 -> 512
        self.fc1 = nn.Linear(
            # 코드 작성
            in_features=128 * 4 * 4,
            out_features=512

        )
        # 파라미터 수: (2048 + 1) * 512 = 1,049,088개

        # Dropout: 과적합 방지 (학습 시 50% 뉴런 랜덤 비활성화)
        self.dropout = nn.Dropout(p=0.5)

        # Fully Connected Layer 2: 512 -> 10 (출력층)
        self.fc2 = nn.Linear(
            in_features=512,
            out_features=num_classes   # 클래스 수 (0-9 숫자)
        )
        # 파라미터 수: (512 + 1) * 10 = 5,130개

    def forward(self, x):
        """
        순전파 함수

        Args:
            x (Tensor): 입력 이미지 [batch, 3, 32, 32]

        Returns:
            Tensor: 클래스별 로짓 [batch, 10]
        """
        # ===== Feature Extractor =====

        # Conv Block 1
        x = self.conv1(x)           # [batch, 3, 32, 32] -> [batch, 32, 32, 32]
        x = F.relu(x)               # ReLU 활성화: 음수는 0, 양수는 그대로
        x = self.pool1(x)           # [batch, 32, 32, 32] -> [batch, 32, 16, 16]

        # Conv Block 2
        x = self.conv2(x)           # [batch, 32, 16, 16] -> [batch, 64, 16, 16]
        x = F.relu(x)
        x = self.pool2(x)           # [batch, 64, 16, 16] -> [batch, 64, 8, 8]

        # Conv Block 3
        x = self.conv3(x)           # [batch, 64, 8, 8] -> [batch, 128, 8, 8]
        x = F.relu(x)
        x = self.pool3(x)           # [batch, 128, 8, 8] -> [batch, 128, 4, 4]

        # ===== Classifier =====

        # Flatten: 다차원 텐서를 1차원으로 펼침
        x = x.view(x.size(0), -1)   # [batch, 128, 4, 4] -> [batch, 2048]
        # x.size(0): 배치 크기
        # -1: 나머지 차원을 자동으로 계산 (128*4*4 = 2048)

        # Fully Connected Layer 1
        x = self.fc1(x)             # [batch, 2048] -> [batch, 512]
        x = F.relu(x)               # ReLU 활성화
        x = self.dropout(x)         # Dropout (학습 시에만 적용)

        # Fully Connected Layer 2 (출력층)
        x = self.fc2(x)             # [batch, 512] -> [batch, 10]
        # Softmax는 CrossEntropyLoss에 포함되어 있으므로 여기서는 생략

        return x  # 로짓(logit) 반환

print('CNN 모델 클래스 정의 완료!')

In [ ]:
# 모델 인스턴스 생성 및 디바이스로 이동

model = SVHN_CNN(num_classes=10)  # 10개 클래스 (0-9)
model = model.to(device)  # 모델을 GPU 또는 CPU로 이동

print('모델 생성 완료!')
print(f'모델이 {device}에 로드되었습니다.\n')

# 모델 구조 출력
print('='*60)
print('모델 구조')
print('='*60)
print(model)
print('='*60)

## Section 4: 파라미터 계산 및 모델 요약

In [ ]:
# 모델의 총 파라미터 수 계산 함수

def count_parameters(model):
    """
    모델의 학습 가능한 파라미터 수를 계산

    Args:
        model (nn.Module): PyTorch 모델

    Returns:
        int: 총 파라미터 수
    """
    # p.numel(): 파라미터 텐서의 원소 개수 (number of elements)
    # p.requires_grad: 그래디언트 계산이 필요한지 여부 (학습 가능 여부)
    total = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total

# 레이어별 파라미터 수 계산 및 출력
def print_parameter_details(model):
    """
    레이어별 파라미터 수와 출력 크기를 상세히 출력
    """
    print('\n레이어별 파라미터 수:')
    print('='*70)
    print(f'{"레이어":<20} {"파라미터 수":>15} {"출력 크기":>25}')
    print('-'*70)

    # Conv1 파라미터 계산
    # (kernel_size * kernel_size * in_channels + 1) * out_channels
    conv1_params = (3 * 3 * 3 + 1) * 32
    print(f'{"Conv1":<20} {conv1_params:>15,} {"[B, 32, 32, 32]":>25}')
    print(f'{"Pool1 (MaxPool)":<20} {0:>15,} {"[B, 32, 16, 16]":>25}')

    # Conv2 파라미터 계산
    conv2_params = (3 * 3 * 32 + 1) * 64
    print(f'{"Conv2":<20} {conv2_params:>15,} {"[B, 64, 16, 16]":>25}')
    print(f'{"Pool2 (MaxPool)":<20} {0:>15,} {"[B, 64, 8, 8]":>25}')

    # Conv3 파라미터 계산
    conv3_params = (3 * 3 * 64 + 1) * 128
    print(f'{"Conv3":<20} {conv3_params:>15,} {"[B, 128, 8, 8]":>25}')
    print(f'{"Pool3 (MaxPool)":<20} {0:>15,} {"[B, 128, 4, 4]":>25}')

    print(f'{"Flatten":<20} {0:>15,} {"[B, 2048]":>25}')

    # FC1 파라미터 계산
    # (in_features + 1) * out_features
    fc1_params = (2048 + 1) * 512
    print(f'{"FC1":<20} {fc1_params:>15,} {"[B, 512]":>25}')
    print(f'{"Dropout":<20} {0:>15,} {"[B, 512]":>25}')

    # FC2 파라미터 계산
    fc2_params = (512 + 1) * 10
    print(f'{"FC2":<20} {fc2_params:>15,} {"[B, 10]":>25}')

    print('='*70)

    # 총 파라미터 수
    total = conv1_params + conv2_params + conv3_params + fc1_params + fc2_params
    print(f'{"총 파라미터 수":<20} {total:>15,}')
    print('='*70)

    return total

# 파라미터 수 계산 및 출력
total_params = count_parameters(model)
print(f'\n모델 총 파라미터 수: {total_params:,}개')

# 레이어별 상세 정보 출력
calculated_total = print_parameter_details(model)

# 계산 검증
print(f'\n검증: 계산된 총 파라미터 수 = {calculated_total:,}개')
print(f'      실제 모델 파라미터 수 = {total_params:,}개')
print(f'      일치 여부: {"✓" if calculated_total == total_params else "✗"}')

## Section 5: Hook을 사용한 레이어 출력 Shape 추적

Forward Hook을 사용하여 각 레이어의 출력 크기를 실시간으로 확인합니다.

In [ ]:
# Hook 함수 정의
# Hook: 모델의 중간 레이어 출력을 가로채는 메커니즘

# 레이어별 출력을 저장할 딕셔너리
layer_outputs = OrderedDict()

def register_hooks(model):
    """
    모델의 모든 레이어에 forward hook을 등록

    Args:
        model: PyTorch 모델

    Returns:
        list: hook handle 리스트 (나중에 제거용)
    """
    handles = []  # hook handle을 저장할 리스트

    def hook_fn(module, input, output):
        """
        Forward hook 함수

        Args:
            module: 현재 레이어 객체
            input: 레이어 입력 (튜플)
            output: 레이어 출력 (텐서)
        """
        # 레이어 이름 생성 (클래스 이름 사용)
        layer_name = module.__class__.__name__

        # 동일한 레이어 타입이 여러 개 있을 경우 번호 추가
        count = sum(1 for k in layer_outputs.keys() if layer_name in k)
        if count > 0:
            layer_name = f"{layer_name}_{count+1}"

        # 출력 크기를 딕셔너리에 저장
        layer_outputs[layer_name] = output.shape

    # 모든 하위 모듈에 hook 등록
    for name, module in model.named_modules():
        # 전체 모델 자체는 제외 (하위 레이어만 등록)
        if len(list(module.children())) == 0 and module != model:
            # register_forward_hook: forward pass 시 hook_fn 실행
            handle = module.register_forward_hook(hook_fn)
            handles.append(handle)

    return handles

# Hook 등록
print('모든 레이어에 Hook 등록 중...')
hook_handles = register_hooks(model)
print(f'총 {len(hook_handles)}개의 Hook이 등록되었습니다.\n')

# 더미 입력으로 순전파 실행 (hook 트리거)
# 입력: [배치=1, 채널=3, 높이=32, 너비=32]
dummy_input = torch.randn(1, 3, 32, 32).to(device)
# randn: 표준 정규분포(평균 0, 표준편차 1)에서 랜덤 샘플링

print('더미 입력으로 순전파 실행 중...')
with torch.no_grad():  # 그래디언트 계산 비활성화 (메모리 절약)
    output = model(dummy_input)

print('\n순전파 완료! 각 레이어의 출력 크기:\n')

# 레이어별 출력 크기 출력
print('='*60)
print(f'{"레이어 이름":<25} {"출력 Shape":>30}')
print('-'*60)
print(f'{"입력 이미지":<25} {str(dummy_input.shape):>30}')
print('-'*60)

for layer_name, shape in layer_outputs.items():
    print(f'{layer_name:<25} {str(tuple(shape)):>30}')

print('='*60)

# Hook 제거 (메모리 정리)
print('\nHook 제거 중...')
for handle in hook_handles:
    handle.remove()  # hook 등록 해제
print('모든 Hook이 제거되었습니다.')

# 출력 크기 해석
print('\n출력 크기 해석:')
print('  - 형식: [배치 크기, 채널 수, 높이, 너비]')
print('  - Conv 후: 채널 수 증가, 공간 크기는 padding으로 유지')
print('  - MaxPool 후: 채널 수 유지, 공간 크기 절반으로 축소')
print('  - Flatten 후: [배치 크기, 총 특징 수]로 1차원화')
print('  - FC 후: [배치 크기, 출력 뉴런 수]')

## Section 6: 학습 함수 정의

In [ ]:
# 학습 함수 정의

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    """
    1 에폭 동안 모델을 학습

    Args:
        model: 학습할 모델
        train_loader: 학습 데이터 로더
        criterion: 손실 함수
        optimizer: 옵티마이저
        device: 디바이스 (cuda 또는 cpu)

    Returns:
        tuple: (평균 손실, 정확도)
    """
    model.train()  # 모델을 학습 모드로 설정 (Dropout, BatchNorm 활성화)

    running_loss = 0.0  # 에폭 전체의 손실 누적
    correct = 0  # 맞춘 샘플 수
    total = 0  # 전체 샘플 수

    # tqdm: 진행률 표시줄 생성
    pbar = tqdm(train_loader, desc='학습 중', leave=False)

    for batch_idx, (inputs, labels) in enumerate(pbar):
        # 데이터를 디바이스로 이동
        inputs = inputs.to(device)  # [batch, 3, 32, 32]
        labels = labels.to(device)  # [batch]

        # 그래디언트 초기화 (이전 배치의 그래디언트 제거)
        optimizer.zero_grad()

        # 순전파 (Forward Pass)
        outputs = model(inputs)  # [batch, 10]

        # 손실 계산
        loss = criterion(outputs, labels)
        # CrossEntropyLoss: Softmax + NLL Loss
        # 입력: 로짓(logit), 정답 레이블

        # 역전파 (Backward Pass)
        loss.backward()  # 그래디언트 계산

        # 가중치 업데이트
        optimizer.step()  # 계산된 그래디언트로 파라미터 업데이트

        # 통계 업데이트
        running_loss += loss.item() * inputs.size(0)
        # loss.item(): 텐서에서 스칼라 값 추출
        # inputs.size(0): 현재 배치 크기

        # 예측값 계산
        _, predicted = outputs.max(1)
        # max(1): dim=1(클래스 차원)에서 최댓값과 인덱스 반환
        # _: 최댓값은 무시
        # predicted: 최댓값의 인덱스 (예측 클래스)

        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        # eq(): 예측과 정답이 같으면 True
        # sum(): True의 개수

        # 진행률 표시줄 업데이트
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })

    # 에폭 평균 손실 및 정확도 계산
    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total

    return epoch_loss, epoch_acc

print('학습 함수 정의 완료!')

## Section 7: 평가 함수 정의

In [ ]:
# 평가 함수 정의

def evaluate(model, test_loader, criterion, device):
    """
    테스트 데이터로 모델 평가

    Args:
        model: 평가할 모델
        test_loader: 테스트 데이터 로더
        criterion: 손실 함수
        device: 디바이스

    Returns:
        tuple: (평균 손실, 정확도)
    """
    model.eval()  # 모델을 평가 모드로 설정 (Dropout 비활성화, BatchNorm은 학습 통계 사용)

    running_loss = 0.0
    correct = 0
    total = 0

    # 그래디언트 계산 비활성화 (평가 시에는 역전파 불필요)
    with torch.no_grad():
        pbar = tqdm(test_loader, desc='평가 중', leave=False)

        for inputs, labels in pbar:
            # 데이터를 디바이스로 이동
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 순전파만 수행 (역전파 없음)
            outputs = model(inputs)

            # 손실 계산
            loss = criterion(outputs, labels)

            # 통계 업데이트
            running_loss += loss.item() * inputs.size(0)

            # 예측값 계산
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            # 진행률 업데이트
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })

    # 평균 손실 및 정확도 계산
    avg_loss = running_loss / total
    accuracy = 100.0 * correct / total

    return avg_loss, accuracy

print('평가 함수 정의 완료!')

## Section 8: 학습 설정 및 실행

In [ ]:
# 학습 설정

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()
# CrossEntropyLoss: 다중 클래스 분류의 표준 손실 함수
# 내부적으로 Softmax + Negative Log-Likelihood 결합
# 입력: 로짓(logit), 정답 레이블(정수)

# 옵티마이저 정의
optimizer = optim.Adam(
    model.parameters(),  # 최적화할 파라미터
    lr=0.001,  # 학습률 (learning rate)
    weight_decay=1e-4  # L2 정규화 계수 (과적합 방지)
)
# Adam: Adaptive Moment Estimation
# - 모멘텀과 RMSProp을 결합한 최적화 알고리즘
# - 각 파라미터마다 적응적 학습률 사용
# - 일반적으로 좋은 성능

# 학습 에폭 수
num_epochs = 10
# 에폭(epoch): 전체 학습 데이터를 한 번 완전히 순회

# 학습 히스토리 저장용
train_losses = []  # 에폭별 학습 손실
train_accs = []  # 에폭별 학습 정확도
test_losses = []  # 에폭별 테스트 손실
test_accs = []  # 에폭별 테스트 정확도

print('학습 설정 완료!')
print('='*60)
print('학습 하이퍼파라미터')
print('='*60)
print(f'에폭 수: {num_epochs}')
print(f'배치 크기: {batch_size}')
print(f'학습률: {optimizer.param_groups[0]["lr"]}')
print(f'가중치 감쇠: {optimizer.param_groups[0]["weight_decay"]}')
print(f'옵티마이저: {optimizer.__class__.__name__}')
print(f'손실 함수: {criterion.__class__.__name__}')
print('='*60)

In [ ]:
# 학습 루프 실행

print('\n학습 시작!\n')
print('='*70)

# 최고 정확도 추적 (모델 저장용)
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'\nEpoch [{epoch+1}/{num_epochs}]')
    print('-'*70)

    # 1. 학습 단계
    train_loss, train_acc = train_one_epoch(
        model, train_loader, criterion, optimizer, device
    )

    # 2. 평가 단계
    test_loss, test_acc = evaluate(
        model, test_loader, criterion, device
    )

    # 3. 결과 저장
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)

    # 4. 에폭 결과 출력
    print(f'\n학습 - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%')
    print(f'테스트 - Loss: {test_loss:.4f}, Acc: {test_acc:.2f}%')

    # 5. 최고 성능 모델 저장
    if test_acc > best_acc:
        best_acc = test_acc
        # 모델 가중치 저장
        torch.save(model.state_dict(), 'best_svhn_model.pth')
        # state_dict(): 모델의 모든 파라미터를 딕셔너리로 반환
        print(f'  → 최고 성능 모델 저장! (정확도: {best_acc:.2f}%)')

print('\n')
print('='*70)
print('학습 완료!')
print(f'최고 테스트 정확도: {best_acc:.2f}%')
print('='*70)

## Section 9: 학습 결과 시각화

In [ ]:
# 학습 곡선 그리기

# 에폭 번호 (x축)
epochs_range = range(1, num_epochs + 1)

# 2x1 서브플롯 생성
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 왼쪽 그래프: 손실 곡선
ax1.plot(epochs_range, train_losses, 'b-', label='학습 손실', marker='o')
ax1.plot(epochs_range, test_losses, 'r-', label='테스트 손실', marker='s')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('손실 곡선 (Loss Curve)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 오른쪽 그래프: 정확도 곡선
ax2.plot(epochs_range, train_accs, 'b-', label='학습 정확도', marker='o')
ax2.plot(epochs_range, test_accs, 'r-', label='테스트 정확도', marker='s')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy (%)', fontsize=12)
ax2.set_title('정확도 곡선 (Accuracy Curve)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 최종 성능 요약
print('\n최종 성능 요약:')
print('='*60)
print(f'최종 학습 손실: {train_losses[-1]:.4f}')
print(f'최종 학습 정확도: {train_accs[-1]:.2f}%')
print(f'최종 테스트 손실: {test_losses[-1]:.4f}')
print(f'최종 테스트 정확도: {test_accs[-1]:.2f}%')
print(f'최고 테스트 정확도: {best_acc:.2f}%')
print('='*60)

In [ ]:
# 예측 결과 시각화
# 테스트 데이터에서 예측 결과 확인

# 최고 성능 모델 로드
model.load_state_dict(torch.load('best_svhn_model.pth'))
# load_state_dict(): 저장된 파라미터를 모델에 로드
model.eval()  # 평가 모드

# 테스트 데이터에서 배치 하나 가져오기
dataiter = iter(test_loader)
images, labels = next(dataiter)

# 모델로 예측
images = images.to(device)
with torch.no_grad():
    outputs = model(images)
    _, predicted = outputs.max(1)

# CPU로 이동 및 역정규화
images = images.cpu()
predicted = predicted.cpu()
labels = labels.cpu()

# 16개 샘플 시각화
plt.figure(figsize=(12, 12))
for i in range(16):
    plt.subplot(4, 4, i+1)

    # 이미지 역정규화 및 차원 변환
    img = denormalize(images[i]).numpy().transpose(1, 2, 0)
    plt.imshow(img)

    # 정답과 예측 표시
    true_label = class_names[labels[i]]
    pred_label = class_names[predicted[i]]

    # 정답이면 파란색, 오답이면 빨간색
    color = 'blue' if true_label == pred_label else 'red'
    plt.title(f'실제: {true_label} / 예측: {pred_label}', color=color, fontsize=10)
    plt.axis('off')

plt.suptitle('테스트 데이터 예측 결과 (파란색: 정답, 빨간색: 오답)',
             fontsize=14, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

# 정확도 계산
correct = (predicted == labels).sum().item()
total = labels.size(0)
print(f'\n현재 배치 정확도: {100.*correct/total:.2f}% ({correct}/{total})')

## Section 10: 혼동 행렬 (Confusion Matrix)

In [ ]:
# 혼동 행렬 계산 및 시각화
from sklearn.metrics import confusion_matrix
import seaborn as sns

# 전체 테스트 데이터에 대한 예측
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc='혼동 행렬 계산 중'):
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

# 혼동 행렬 계산
cm = confusion_matrix(all_labels, all_preds)

# 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('예측 레이블', fontsize=12)
plt.ylabel('실제 레이블', fontsize=12)
plt.title('혼동 행렬 (Confusion Matrix)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 클래스별 정확도 계산
print('\n클래스별 정확도:')
print('='*60)
print(f'{"숫자":<10} {"정확도":>15} {"샘플 수":>15}')
print('-'*60)

for i in range(10):
    # 해당 클래스의 정확도 계산
    class_correct = cm[i, i]  # 대각선 원소 (정답)
    class_total = cm[i].sum()  # 해당 행의 합 (전체)
    class_acc = 100.0 * class_correct / class_total if class_total > 0 else 0
    print(f'{class_names[i]:<10} {class_acc:>14.2f}% {class_total:>15}')

print('='*60)

## Section 11: 실습 요약 및 개선 아이디어

In [ ]:
# 실습 요약 출력

print('\n')
print('='*70)
print('CNN 모델 빌드 실습 요약')
print('='*70)

print('\n1. 데이터셋: SVHN (Street View House Numbers)')
print(f'   - 학습 데이터: {len(train_dataset):,}개')
print(f'   - 테스트 데이터: {len(test_dataset):,}개')
print(f'   - 클래스 수: 10개 (0-9 숫자)')

print('\n2. 모델 아키텍처:')
print('   Feature Extractor:')
print('     - Conv1(32) + ReLU + MaxPool')
print('     - Conv2(64) + ReLU + MaxPool')
print('     - Conv3(128) + ReLU + MaxPool')
print('   Classifier:')
print('     - Flatten')
print('     - FC1(2048->512) + ReLU + Dropout(0.5)')
print('     - FC2(512->10)')

print(f'\n3. 총 파라미터 수: {total_params:,}개')

print('\n4. 학습 설정:')
print(f'   - 에폭: {num_epochs}')
print(f'   - 배치 크기: {batch_size}')
print(f'   - 옵티마이저: Adam (lr=0.001)')
print(f'   - 손실 함수: CrossEntropyLoss')

print('\n5. 최종 성능:')
print(f'   - 최고 테스트 정확도: {best_acc:.2f}%')
print(f'   - 최종 테스트 정확도: {test_accs[-1]:.2f}%')
print(f'   - 최종 테스트 손실: {test_losses[-1]:.4f}')

print('\n6. 주요 학습 내용:')
print('   ✓ CNN 아키텍처 설계 (Feature Extractor + Classifier 분리)')
print('   ✓ 파라미터 수 계산 및 모델 크기 파악')
print('   ✓ Hook을 사용한 레이어 출력 shape 추적')
print('   ✓ 데이터 증강을 통한 성능 향상')
print('   ✓ 학습 곡선 및 혼동 행렬 분석')

print('\n')
print('='*70)
print('실습 완료!')
print('='*70)

## 개선 아이디어 및 추가 실습

### 1. 더 깊은 네트워크
- Conv 블록을 4-5개로 증가
- Batch Normalization 추가
- Residual Connection 도입

### 2. 정규화 기법
- Batch Normalization 추가
- Layer Normalization 실험
- L2 정규화 강도 조정

### 3. 데이터 증강 강화
- RandomRotation 추가
- RandomErasing 적용
- Mixup, CutMix 기법

### 4. 학습률 스케줄링
- StepLR: 일정 에폭마다 학습률 감소
- CosineAnnealingLR: Cosine 함수로 학습률 조정
- ReduceLROnPlateau: 성능 정체 시 학습률 감소

### 5. 앙상블
- 여러 모델의 예측 결합
- Test Time Augmentation (TTA)

### 6. 전이학습
- 사전학습된 ResNet, VGG 모델 활용
- Fine-tuning 실험

### 7. 하이퍼파라미터 튜닝
- 배치 크기 실험 (64, 128, 256)
- 학습률 범위 탐색 (1e-5 ~ 1e-2)
- Dropout 비율 조정 (0.3, 0.5, 0.7)

### 8. 시각화 확장
- Feature Map 시각화
- Grad-CAM (Class Activation Mapping)
- 필터 시각화

